In [5]:
from datasets import load_dataset
import tiktoken

In [4]:
ds = load_dataset("open-r1/OpenR1-Math-220k")
train = ds["train"]
train

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset({
    features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages'],
    num_rows: 93733
})

In [11]:
train["correctness_math_verify"][0]

[True, False]

In [6]:
tokenizer = tiktoken.get_encoding("cl100k_base")

In [23]:
# Filter the dataset using correctness_math_verify
two_or_more = train.filter(lambda x: sum(x["correctness_math_verify"]) >= 2)

Filter:   0%|          | 0/93733 [00:00<?, ? examples/s]

In [24]:
# Function to filter the dataset
def filter_generations(row):
    # Ensure that both shortest and longest generations are correct
    correct_generations = [
        i for i, correct in enumerate(row['correctness_math_verify']) if correct
    ]
    
    if len(correct_generations) < 2:
        # If there are not at least two correct generations, skip this row
        return None
    
    # Get the lengths of the correct generations
    correct_lengths = [len(tokenizer.encode(row['generations'][i])) for i in correct_generations]
    min_id = correct_lengths.index(min(correct_lengths))
    max_id = correct_lengths.index(max(correct_lengths))
    
    row['generations'] = [row['generations'][min_id], row['generations'][max_id]]
    row['generations_length'] = [correct_lengths[min_id], correct_lengths[max_id]]
    
    return row

# Apply the filter to the dataset
filtered_dataset = two_or_more.map(filter_generations)

# Show the filtered dataset
print(filtered_dataset)

Map:   0%|          | 0/54194 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages', 'generations_length'],
    num_rows: 54194
})


In [25]:
filtered_dataset = filtered_dataset.filter(lambda x: x["generations_length"][0] <= 8162 and x["generations_length"][1] <= 8162)
print(filtered_dataset)

Filter:   0%|          | 0/54194 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages', 'generations_length'],
    num_rows: 41896
})
